In [1]:
import pandas as pd
from transformers import pipeline

In [2]:
education = pd.read_csv("/Users/shu/training_python_datascince/kaggle_sentimental_analysis/Sentiment_dataset/Education.csv")
finance = pd.read_csv("/Users/shu/training_python_datascince/kaggle_sentimental_analysis/Sentiment_dataset/Finance.csv")
politics = pd.read_csv("/Users/shu/training_python_datascince/kaggle_sentimental_analysis/Sentiment_dataset/Politics.csv")
sports = pd.read_csv("/Users/shu/training_python_datascince/kaggle_sentimental_analysis/Sentiment_dataset/Sports.csv")

In [3]:
print(education.Label.value_counts(), '\n\n', education.isna().any())

positive    26
negative    26
Name: Label, dtype: int64 

 Text     False
Label    False
dtype: bool


In [4]:
print(finance.Label.value_counts(), '\n\n', finance.isna().any())

positive    34
negative    14
Name: Label, dtype: int64 

 Text     False
Label    False
dtype: bool


In [5]:
print(politics.Label.value_counts(), '\n\n', politics.isna().any())

negative    28
positive    25
Name: Label, dtype: int64 

 Text     False
Label    False
dtype: bool


In [6]:
print(sports.Label.value_counts(), '\n\n', sports.isna().any())

positive    28
negative    28
Name: Label, dtype: int64 

 Text     False
Label    False
dtype: bool


In [7]:
df = pd.concat([education, politics, sports, finance])

In [8]:
df = df.sample(frac=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 49 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    209 non-null    object
 1   Label   209 non-null    object
dtypes: object(2)
memory usage: 4.9+ KB


In [10]:
df.isna().any()

Text     False
Label    False
dtype: bool

In [11]:
df.duplicated().any()

False

In [12]:
df.head()

,Text,Label
49,Educational inequities persist despite efforts...,negative
47,Financial bubbles can arise when asset prices ...,negative
20,Financial regulators face challenges in keepin...,negative
19,Sports events can unite people from diverse ba...,positive
35,The concept of risk-adjusted returns is essent...,positive


In [13]:
df_4 = [education, politics, sports, finance]

In [14]:
model_names = [
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    "cardiffnlp/twitter-roberta-base-sentiment-latest",
    "siebert/sentiment-roberta-large-english"
]

models = []

for model_name in model_names:
    model = pipeline(model=model_name)
    models.append(model)

/Users/shu/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to b

In [22]:
metrics = ['Model','Accuracy','Precision','Recall','F1_Score','Hamming_Loss', "Cohen\'s_Kappa", "Log_Loss"]

In [23]:
education_result = pd.DataFrame(columns = metrics, index=[0,1,2,3])
sports_result = pd.DataFrame(columns = metrics, index=[0,1,2,3])
politics_result = pd.DataFrame(columns = metrics, index=[0,1,2,3])
finance_result = pd.DataFrame(columns = metrics, index=[0,1,2,3])

In [24]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            hamming_loss, cohen_kappa_score, log_loss)

In [25]:
for idx, theme in enumerate(df_4, start=1):

    for i in range(len(models)):

        model_pred=[]

        # Predicted labels
        for str_ in theme['Text']:
            model_pred.append(1 if models[i](str_)[0]['label'].lower()=='positive' else 0)
        
        lable = theme['Label'].apply(lambda x: {True: 1, False: 0}[x=='positive'])
        
        # Metrics
        accuracy = accuracy_score(lable, model_pred)
        precision = precision_score(lable, model_pred)
        recall = recall_score(lable, model_pred)
        f1 = f1_score(lable, model_pred)
        hamming = hamming_loss(lable, model_pred)
        kappa = cohen_kappa_score(lable, model_pred)
        ll = log_loss(lable, model_pred)

        if idx==1:
            # Education
            education_result.loc[i] = [f"Model {i+1}", accuracy, precision, recall, f1, hamming, kappa, ll]
        elif idx==2:
            # Sports
            sports_result.loc[i] = [f"Model {i+1}", accuracy, precision, recall, f1, hamming, kappa, ll]
        elif idx==3:
            # Politics
            politics_result.loc[i] = [f"Model {i+1}", accuracy, precision, recall, f1, hamming, kappa, ll]
        else:
            # Finance
            finance_result.loc[i] = [f"Model {i+1}", accuracy, precision, recall, f1, hamming, kappa, ll]

In [26]:
education_result

,Model,Accuracy,Precision,Recall,F1_Score,Hamming_Loss,Cohen's_Kappa,Log_Loss
0,Model 1,0.576923,0.611111,0.423077,0.5,0.423077,0.153846,15.249238
1,Model 2,0.634615,0.6,0.807692,0.688525,0.365385,0.269231,13.169796
2,Model 3,0.634615,0.888889,0.307692,0.457143,0.365385,0.269231,13.169796
3,Model 4,0.673077,0.714286,0.576923,0.638298,0.326923,0.346154,11.783502


In [27]:
sports_result

,Model,Accuracy,Precision,Recall,F1_Score,Hamming_Loss,Cohen's_Kappa,Log_Loss
0,Model 1,0.849057,0.947368,0.72,0.818182,0.150943,0.693198,5.440551
1,Model 2,0.754717,0.772727,0.68,0.723404,0.245283,0.504673,8.840896
2,Model 3,0.54717,1.0,0.04,0.076923,0.45283,0.042169,16.321654
3,Model 4,0.924528,0.92,0.92,0.92,0.075472,0.848571,2.720276


In [28]:
politics_result

,Model,Accuracy,Precision,Recall,F1_Score,Hamming_Loss,Cohen's_Kappa,Log_Loss
0,Model 1,0.839286,0.827586,0.857143,0.842105,0.160714,0.678571,5.79273
1,Model 2,0.803571,0.742857,0.928571,0.825397,0.196429,0.607143,7.080003
2,Model 3,0.892857,1.0,0.785714,0.88,0.107143,0.785714,3.86182
3,Model 4,0.910714,0.896552,0.928571,0.912281,0.089286,0.821429,3.218183


In [29]:
finance_result

,Model,Accuracy,Precision,Recall,F1_Score,Hamming_Loss,Cohen's_Kappa,Log_Loss
0,Model 1,0.8125,0.903226,0.823529,0.861538,0.1875,0.573123,6.758185
1,Model 2,0.770833,0.870968,0.794118,0.830769,0.229167,0.478261,8.260004
2,Model 3,0.416667,1.0,0.176471,0.3,0.583333,0.111111,21.025464
3,Model 4,0.895833,0.914286,0.941176,0.927536,0.104167,0.742489,3.754547
